In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import genesis
nltk.download('genesis')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
genesis_ic = wn.ic(genesis, False, 0.0)

import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer 
from nltk.stem import SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from sklearn.metrics import roc_auc_score

[nltk_data] Downloading package genesis to /root/nltk_data...
[nltk_data]   Unzipping corpora/genesis.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [0]:
location = "/content/drive/Shared drives/CMPE 257: Machine Learning/AlterusVera-Datasets/liar_dataset/"
location1= "/content/drive/Shared drives/CMPE 257: Machine Learning/AlterusVera-Datasets/"
location_fake_nonfake_processed = "/content/drive/Shared drives/CMPE 257: Machine Learning/AlterusVera-Datasets/fake-news-kaggle/"

In [4]:
data_liar_liar = pd.read_csv(location+'liarliar_lda_sentiment_processed_data.csv', low_memory =False, encoding = 'utf-8')
data_liar_liar.head(3)

,Unnamed: 0,headline_text,processed_headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,context,encoded_label,tokenized,lda_topics,negative,neutral,positive,compound,subject_codes,speaker_codes,partyaffiliation_codes,speakerjobtitle_codes,context_codes,lda_topic_codes
0,0,Says the Annies List political group supports ...,nanni list polit group support third trimest a...,abortion,dwayne-bohac,State representative,Texas,republican,a mailer,5,"['nanni', 'list', 'polit', 'group', 'support',...",polit third support nanni administr trimest li...,0.0,1.000,0.000,0.0000,31,868,20,930,1430,2950
1,1,When did the decline of coal start? It started...,declin coal start start natur took start begin...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,a floor speech.,3,"['declin', 'coal', 'start', 'start', 'natur', ...",administr polit abort list trimest third suppo...,0.0,0.794,0.206,0.0772,2810,2652,6,928,1257,811
2,2,"Hillary Clinton agrees with John McCain ""by vo...",hillari clinton agre john mccain vote give geo...,foreign-policy,barack-obama,President,Illinois,democrat,Denver,2,"['hillari', 'clinton', 'agr', 'john', 'mccain'...",polit trimest group administr third support na...,0.0,0.597,0.403,0.4019,3139,197,6,686,87,2983


In [5]:
data_fake_nonfake_processed = pd.read_csv(location_fake_nonfake_processed+'all_data_coverage_condensed_binary_20k.csv', low_memory =False, encoding = 'utf-8')
data_fake_nonfake_processed.head(3)

,Unnamed: 0,date,title,text,type,text_clean,topics,topic_0,topic_1,date_int,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,...,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,Coverage,tokenized,lda_topics
0,0,2016-10-26,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,0,"['print', 'pay', 'back', 'money', 'plus', 'int...","['government ', 'group']","[-1.3655588626861572, 1.3803073167800903, -0.8...","[-1.3655588626861572, 1.3803073167800903, -0.8...",201610,-1.365559,1.380307,-0.887381,1.158309,-0.758152,0.796280,-2.433508,1.571465,-0.744461,-1.641097,-0.106363,1.827266,-0.823118,2.154561,2.215422,1.272591,2.206310,4.717751,1.270385,-1.959910,0.797869,-0.629742,-1.037067,0.251305,0.220812,0.061147,-0.825234,2.738059,-0.652101,1.201536,...,-1.965121,-1.226513,0.255262,1.285010,2.642656,-3.255653,2.110342,0.344072,-1.635603,3.002436,-1.164981,1.879149,-0.248772,-1.684341,-0.685247,-1.515965,-0.157808,0.556845,0.537664,0.206301,-1.509730,-1.629651,-0.235045,-0.160433,-0.453009,1.531871,0.038206,-3.259110,-2.416788,0.081683,-0.345003,2.535383,-1.430031,-1.408068,-0.519396,-2.536549,2.231460,0,"['print', 'pay', 'back', 'money', 'plu', 'inte...",commit came bust deport entir control case ben...
1,1,2016-10-29,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,0,"['attorney', 'general', 'loretta', 'lynch', 'p...","['lynch ', 'cash']","[3.6489102840423584, 1.46665358543396, -0.1478...","[3.6489102840423584, 1.46665358543396, -0.1478...",201610,3.648910,1.466654,-0.147848,-0.491413,2.218158,0.947100,-1.474590,-0.298721,-1.471738,-2.181059,2.570954,-0.972448,-0.880758,0.137702,-0.813711,-1.952407,2.438278,-0.596132,3.182331,-0.901922,0.063674,0.241279,-0.062920,-0.018689,3.422373,1.017810,4.120409,-2.412775,1.668786,-1.544768,...,-3.049003,1.457955,-1.921744,-1.180705,1.880241,-1.504865,-1.224527,1.717557,1.218073,1.085071,-2.645739,-0.833238,-1.316788,1.044427,-1.011497,-2.125757,-1.285155,1.602046,0.692449,-0.363378,-1.984114,-0.693873,2.731153,0.520502,0.439609,-1.118309,0.579842,0.037475,-0.422936,-0.451379,1.614204,-0.039054,2.467978,1.105344,1.540882,0.767650,0.423304,0,"['attorney', 'gener', 'loretta', 'lynch', 'ple...",control case asap commit entir back came depor...
2,2,2016-10-31,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,0,"['red', 'state', 'fox', 'news', 'sunday', 'rep...","['weiner ', 'form']","[1.0462281703948975, -0.7262989282608032, -0.6...","[1.0462281703948975, -0.7262989282608032, -0.6...",201610,1.046228,-0.726299,-0.609664,0.864581,-0.592420,2.446698,-1.693561,-1.588403,0.303621,-1.117802,1.920915,2.911680,-0.161205,3.105719,-1.864564,-1.734798,2.316164,0.524890,1.768682,3.503988,0.302414,0.795189,-0.082322,0.093117,3.223644,1.611049,-0.099279,-0.391072,0.191230,-0.994695,...,0.428779,0.526036,-0.486134,0.875889,3.693366,2.095969,0.338016,1.311571,-0.372086,1.865512,-1.204279,-3.674162,-0.701656,-2.376498,-2.346642,0.374807,-3.393143,-0.890868,-0.897993,-0.037599,0.933873,1.821212,1.309603,-2.444696,2.501420,-3.269751,3.284004,-1.141671,1.005922,0.492880,4.423038,-0.519755,-0.222670,2.393956,0.930297,1.260847,0.077982,0,"['red', 'state', 'fox', 'news', 'sunday', 'rep...",asap control bust entir deport commit case cam...


## KNN_NLC_Classifer

In [0]:
from nltk import word_tokenize, sent_tokenize 
from nltk.corpus import wordnet as wn

class KNN_NLC_Classifer():
    def __init__(self, k=1, distance_type = 'path'):
        self.k = k
        self.distance_type = distance_type

    # This function is used for training
    def fit(self, x_train, y_train):
        self.x_train = x_train
        self.y_train = y_train

    # This function runs the K(1) nearest neighbour algorithm and
    # returns the label with closest match. 
    def predict(self, x_test):
        #print('predict ')

        self.x_test = x_test
        #print(x_test)
        y_predict = []
        #print(self.x_train.shape[0])
        for i in range(len(x_test)):
            #print(len(x_test))
            if i % 500 == 0: print('processing ',i,' of ',len(x_test))
            max_sim = 0
            max_index = 0
            MAX = 1.0
            #print('x1 : ',x_test[i]);
            for j in range(self.x_train.shape[0]):
                #print(i,j)
                #print('x1 x2 ',x_test[i], self.x_train['lda_topics'][j])
                temp = self.document_similarity(x_test[i], self.x_train['lda_topics'][j])
                print('temp-distance: ',temp)
                if temp > max_sim:
                    max_sim = temp
                    max_index = j
                if max_sim == MAX:
                    break;    
            #print('max_inde: ',max_index)
            #print(self.y_train.loc[max_index])       
            #print(self.y_train.loc[max_index]['type'])    
            y_predict.append(self.y_train.loc[max_index]['encoded_label'])
        return y_predict


    def convert_tag(self, tag):
        """Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""
        tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
        try:
            return tag_dict[tag[0]]
        except KeyError:
            return None


    def doc_to_synsets(self, doc):
        """
            Returns a list of synsets in document.
            Tokenizes and tags the words in the document doc.
            Then finds the first synset for each word/tag combination.
        If a synset is not found for that combination it is skipped.

        Args:
            doc: string to be converted

        Returns:
            list of synsets
        """
        tokens = word_tokenize(doc+' ')
        
        l = []
        tags = nltk.pos_tag([tokens[0] + ' ']) if len(tokens) == 1 else nltk.pos_tag(tokens)
        
        for token, tag in zip(tokens, tags):
            syntag = self.convert_tag(tag[1])
            syns = wn.synsets(token, syntag)
            if (len(syns) > 0):
                l.append(syns[0])
        return l  

    def document_similarity(self,doc1, doc2):
          """Finds the symmetrical similarity between doc1 and doc2"""

          synsets1 = self.doc_to_synsets(doc1)
          synsets2 = self.doc_to_synsets(doc2)
         
          
          return (self.similarity_score(synsets1, synsets2,'wup') + self.similarity_score(synsets2, synsets1,'wup')) / 2

    def similarity_score(self, s1, s2, distance_type = 'path'):
          """
          Calculate the normalized similarity score of s1 onto s2
          For each synset in s1, finds the synset in s2 with the largest similarity value.
          Sum of all of the largest similarity values and normalize this value by dividing it by the
          number of largest similarity values found.

          Args:
              s1, s2: list of synsets from doc_to_synsets

          Returns:
              normalized similarity score of s1 onto s2
          """
          s1_largest_scores = []

          for i, s1_synset in enumerate(s1, 0):
              max_score = 0
              for s2_synset in s2:
                  if distance_type == 'path':
                      score = s1_synset.path_similarity(s2_synset, simulate_root = False)
                  else:
                      score = s1_synset.wup_similarity(s2_synset)
                      if score != None:
                         
                        if score > max_score:
                            max_score = score
              
              if max_score != 0:
                  s1_largest_scores.append(max_score)
          
          mean_score = np.mean(s1_largest_scores)
          

          return mean_score  
    

# Prepare data

train can be ur complete LL dataset
* type is label

In [0]:
data_liar_liar = data_liar_liar [:1000]

In [8]:
X_train = data_liar_liar[data_liar_liar['lda_topics'] != '']
print(X_train.shape)

X_train = X_train[['lda_topics']]
print(X_train.shape)
Y_train = data_liar_liar[['encoded_label']]

(1000, 22)
(1000, 1)


Test is your new dataset after doing LDA topics 

In [9]:
data_liar_liar.head(1)

,Unnamed: 0,headline_text,processed_headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,context,encoded_label,tokenized,lda_topics,negative,neutral,positive,compound,subject_codes,speaker_codes,partyaffiliation_codes,speakerjobtitle_codes,context_codes,lda_topic_codes
0,0,Says the Annies List political group supports ...,nanni list polit group support third trimest a...,abortion,dwayne-bohac,State representative,Texas,republican,a mailer,5,"['nanni', 'list', 'polit', 'group', 'support',...",polit third support nanni administr trimest li...,0.0,1.0,0.0,0.0,31,868,20,930,1430,2950


In [13]:
data_fake_nonfake_processed.head(1)

,Unnamed: 0,date,title,text,type,text_clean,topics,topic_0,topic_1,date_int,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,...,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,Coverage,tokenized,lda_topics
0,0,2016-10-26,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,0,"['print', 'pay', 'back', 'money', 'plus', 'int...","['government ', 'group']","[-1.3655588626861572, 1.3803073167800903, -0.8...","[-1.3655588626861572, 1.3803073167800903, -0.8...",201610,-1.365559,1.380307,-0.887381,1.158309,-0.758152,0.79628,-2.433508,1.571465,-0.744461,-1.641097,-0.106363,1.827266,-0.823118,2.154561,2.215422,1.272591,2.20631,4.717751,1.270385,-1.95991,0.797869,-0.629742,-1.037067,0.251305,0.220812,0.061147,-0.825234,2.738059,-0.652101,1.201536,...,-1.965121,-1.226513,0.255262,1.28501,2.642656,-3.255653,2.110342,0.344072,-1.635603,3.002436,-1.164981,1.879149,-0.248772,-1.684341,-0.685247,-1.515965,-0.157808,0.556845,0.537664,0.206301,-1.50973,-1.629651,-0.235045,-0.160433,-0.453009,1.531871,0.038206,-3.25911,-2.416788,0.081683,-0.345003,2.535383,-1.430031,-1.408068,-0.519396,-2.536549,2.23146,0,"['print', 'pay', 'back', 'money', 'plu', 'inte...",commit came bust deport entir control case ben...


In [10]:
test = data_fake_nonfake_processed[data_fake_nonfake_processed['lda_topics'] != ''][:1000]
print(test.shape)

(1000, 213)


In [0]:
test_lines = test[test['lda_topics'] != '']; # get all articles with LDA topics != empty
test_lines = test_lines['lda_topics'].tolist() # convert all articles dataframe to list 

   


Below code is optional, if u want to do line of line preprocessing for the new dataset. If u have already completed preprocessing, skip the below code cell

In [13]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [0]:
import re
stop_words = stopwords.words('english')

test_corpus = []
ps = PorterStemmer()
for i in range(len(test_lines)):
    #print(test_lines[i])
    review = re.sub('[^a-zA-Z]', ' ', test_lines[i])
    review = review.lower()
    review = review.split()

    review = [word for word in review if not word in stop_words]
    review = ' '.join(review)
    test_corpus.append(review)

In [0]:
classifier = KNN_NLC_Classifer(k=1, distance_type='wcp')
classifier.fit(X_train, Y_train) # 


the below code will return the label of document which is close to the given corpus 

In [16]:
y_pred_final = classifier.predict(test_lines)
# if your given article is close to article number 10 in LL dataset, u will get the label of article number 10 eg., 'Half true'


Streaming output truncated to the last 5000 lines.
temp-distance:  0.45654456654456654
temp-distance:  0.45654456654456654
temp-distance:  0.45654456654456654
temp-distance:  0.45654456654456654
temp-distance:  0.45654456654456654
temp-distance:  0.45654456654456654
temp-distance:  0.45654456654456654
temp-distance:  0.45654456654456654
temp-distance:  0.45654456654456654
temp-distance:  0.45654456654456654
temp-distance:  0.45654456654456654
temp-distance:  0.45654456654456654
temp-distance:  0.45654456654456654
temp-distance:  0.45654456654456654
temp-distance:  0.45654456654456654
temp-distance:  0.45654456654456654
temp-distance:  0.45654456654456654
temp-distance:  0.45654456654456654
temp-distance:  0.45654456654456654
temp-distance:  0.45654456654456654
temp-distance:  0.45654456654456654
temp-distance:  0.45654456654456654
temp-distance:  0.45654456654456654
temp-distance:  0.45654456654456654
temp-distance:  0.45654456654456654
temp-distance:  0.45654456654456654
temp-distance

In [17]:
np.shape(y_pred_final)


(1000,)

In [0]:
test['encoded_liarliar_label'] = y_pred_final

In [37]:
test.head(2)

,Unnamed: 0,date,title,text,type,text_clean,topics,topic_0,topic_1,date_int,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,...,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,Coverage,tokenized,lda_topics,encoded_label_new,encoded_liarliar_label
0,0,2016-10-26,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,0,"['print', 'pay', 'back', 'money', 'plus', 'int...","['government ', 'group']","[-1.3655588626861572, 1.3803073167800903, -0.8...","[-1.3655588626861572, 1.3803073167800903, -0.8...",201610,-1.365559,1.380307,-0.887381,1.158309,-0.758152,0.79628,-2.433508,1.571465,-0.744461,-1.641097,-0.106363,1.827266,-0.823118,2.154561,2.215422,1.272591,2.206310,4.717751,1.270385,-1.959910,0.797869,-0.629742,-1.037067,0.251305,0.220812,0.061147,-0.825234,2.738059,-0.652101,1.201536,...,0.255262,1.285010,2.642656,-3.255653,2.110342,0.344072,-1.635603,3.002436,-1.164981,1.879149,-0.248772,-1.684341,-0.685247,-1.515965,-0.157808,0.556845,0.537664,0.206301,-1.509730,-1.629651,-0.235045,-0.160433,-0.453009,1.531871,0.038206,-3.259110,-2.416788,0.081683,-0.345003,2.535383,-1.430031,-1.408068,-0.519396,-2.536549,2.231460,0,"['print', 'pay', 'back', 'money', 'plu', 'inte...",commit came bust deport entir control case ben...,3,3
1,1,2016-10-29,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,0,"['attorney', 'general', 'loretta', 'lynch', 'p...","['lynch ', 'cash']","[3.6489102840423584, 1.46665358543396, -0.1478...","[3.6489102840423584, 1.46665358543396, -0.1478...",201610,3.648910,1.466654,-0.147848,-0.491413,2.218158,0.94710,-1.474590,-0.298721,-1.471738,-2.181059,2.570954,-0.972448,-0.880758,0.137702,-0.813711,-1.952407,2.438278,-0.596132,3.182331,-0.901922,0.063674,0.241279,-0.062920,-0.018689,3.422373,1.017810,4.120409,-2.412775,1.668786,-1.544768,...,-1.921744,-1.180705,1.880241,-1.504865,-1.224527,1.717557,1.218073,1.085071,-2.645739,-0.833238,-1.316788,1.044427,-1.011497,-2.125757,-1.285155,1.602046,0.692449,-0.363378,-1.984114,-0.693873,2.731153,0.520502,0.439609,-1.118309,0.579842,0.037475,-0.422936,-0.451379,1.614204,-0.039054,2.467978,1.105344,1.540882,0.767650,0.423304,0,"['attorney', 'gener', 'loretta', 'lynch', 'ple...",control case asap commit entir back came depor...,3,3


In [0]:
test.to_csv(location+"amalgamation_fakenws_allnews_liarliar_data.csv", sep=',')

1. load LDa_merged.csv
2. X_train = df of LDA_topics [liar liar]
3. Y_train = df of label [liar liar]
4. load fake news
5. compute lda on fake news [have lda_topics column]
6. take lda_topics column and convert it into list
7. list of topics = test_lines
8. y_pred_final >> Test['type'] = y_pred_final
9. pd.concat train,test